<a href="https://colab.research.google.com/github/anjanpa/Regression-House-Price-prediction-kaggle-/blob/master/House_Price_Regression_Hypertuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Getting our tools ready
import pandas as pd
import numpy as plt
import numpy as np


In [2]:
## IMporting the preprocessed data uploaded from jupyter 
train_preprocessed=pd.read_csv("/content/drive/My Drive/house_price_preprocessed_data.csv")

In [3]:
train_preprocessed.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1350,70,5,50.0,5250,2,2,4,4,1,5,1,18,3,3,1,6,8,5,1872,1987,2,2,9,9,3,0.0,4,3,1,4,1,4,4,259,6,0,425,684,5,...,1,5,938,1215,205,2358,0,0,2,0,4,1,4,8,7,0,0,0,1980.0,0,0,0,0,0,3,0,54,20,0,0,0,0,0,0,0,12,2008,9,5,122000
1,1138,50,4,54.0,6342,2,0,4,4,1,5,1,20,2,3,1,1,5,8,1875,1996,2,2,13,14,3,0.0,4,3,2,4,4,4,6,0,6,0,780,780,2,...,1,5,780,240,0,1020,0,0,1,0,2,1,4,6,7,0,0,0,1980.0,0,0,0,0,0,1,0,0,176,0,0,0,0,0,0,0,5,2010,9,5,94000
2,748,70,5,65.0,11700,2,2,1,4,1,1,1,18,3,3,1,6,7,7,1880,2003,5,2,12,13,3,0.0,3,5,5,4,1,4,6,0,6,0,1240,1240,3,...,1,5,1320,1320,0,2640,0,0,1,1,4,1,3,8,7,1,3,6,1950.0,3,4,864,5,5,1,181,0,386,0,0,0,0,0,0,0,5,2009,9,5,265979
3,305,75,5,87.0,18386,2,0,4,4,1,5,1,18,3,3,1,4,7,9,1880,2002,2,2,6,6,3,0.0,4,5,1,4,4,4,6,0,6,0,1470,1470,2,...,2,5,1675,1818,0,3493,0,0,3,0,3,1,3,10,7,1,1,2,2003.0,3,3,870,5,5,3,302,0,0,0,0,0,0,0,0,0,5,2008,9,5,295000
4,1133,70,5,90.0,9900,2,0,4,4,1,5,1,18,3,3,1,6,6,4,1880,1950,2,2,14,15,3,0.0,4,5,1,4,4,3,6,0,6,0,1008,1008,3,...,2,5,1178,1032,0,2210,0,0,2,0,5,1,2,8,7,0,0,6,1930.0,3,1,205,2,5,1,0,48,0,0,0,0,0,0,0,0,5,2007,9,5,117500


# Evaluating results in different MODELS

In [7]:
## Splitting the data into train and test
np.random.seed(100)
from sklearn.model_selection import train_test_split
X=train_preprocessed.drop("SalePrice",axis=1)
y=train_preprocessed["SalePrice"]
X_train,X_valid,y_train,y_valid=train_test_split(X,
                                                 y,
                                                 test_size=0.2,
                                                 shuffle=True)

In [8]:
## Import the models
## IMporting the models for regression to experiment on those 
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [9]:
## Making a dictionary of models
models={"LinearRegression":LinearRegression(),
       "Lasso":Lasso(),
       "Ridge":Ridge(),
       "RandomForestRegression":RandomForestRegressor(),
       "Gradient":GradientBoostingRegressor(),
       "AdaBoostRegressor":AdaBoostRegressor(),
       "SVR":SVR(),
       "DecisionTreeRegressor":DecisionTreeRegressor()}

In [12]:
## Making a function to fit and score on the models
from sklearn.metrics import mean_absolute_error
def fit_and_score(models,X_train,y_train,X_valid,y_valid):
    results={}
    for model_name,model in models.items():
        model.fit(X_train,y_train)
        results[model_name+" accuracy"]=model.score(X_valid,y_valid)
        y_preds=model.predict(X_valid)
        results[model_name+" mean_absolute_error"]=mean_absolute_error(y_valid,y_preds)

    return results
        

In [15]:
## Let's evaluate the function
%%time
result_1=fit_and_score(models,X_train,y_train,X_valid,y_valid)
result_1

CPU times: user 2.98 s, sys: 139 ms, total: 3.12 s
Wall time: 2.97 s


In [16]:
result_1

{'AdaBoostRegressor accuracy': 0.8288756579802785,
 'AdaBoostRegressor mean_absolute_error': 22086.153119072078,
 'DecisionTreeRegressor accuracy': 0.7467040846726105,
 'DecisionTreeRegressor mean_absolute_error': 24298.87671232877,
 'Gradient accuracy': 0.8819786831117113,
 'Gradient mean_absolute_error': 16262.283475624627,
 'Lasso accuracy': 0.7963561438273482,
 'Lasso mean_absolute_error': 19303.689239995423,
 'LinearRegression accuracy': 0.7961041368744275,
 'LinearRegression mean_absolute_error': 19309.667603418857,
 'RandomForestRegression accuracy': 0.8800196812918526,
 'RandomForestRegression mean_absolute_error': 17042.85609589041,
 'Ridge accuracy': 0.8270878164693442,
 'Ridge mean_absolute_error': 19069.553170312112,
 'SVR accuracy': -0.058692609181610685,
 'SVR mean_absolute_error': 53341.10468022511}

#### NOTE: 
**SVR is discarded due to low metric obtained and is very hard to improve**

## Hyperparameter tuning on different models using GridSearchCV

## 1.RandomForestRegressor

In [17]:
from sklearn.model_selection import GridSearchCV


In [20]:
## The parameter's values have been set using thorough research
np.random.seed(100)
randomforest_grid={"criterion":["mae"],
           "n_estimators":[100,150,200],
           "min_samples_split":[2,4],
           "max_features":[None,"sqrt"]}
RandomForest_grid=GridSearchCV(RandomForestRegressor(),
                               param_grid=randomforest_grid,
                               cv=5,
                               verbose=True)

In [21]:
## Create a function to check metrics of each model
def check_metrics(model):
    model.fit(X_train,y_train)
    print(model.best_params_)
    print("accuracy",model.score(X_valid,y_valid))
    print("mae",mean_absolute_error(y_valid,model.predict(X_valid)))

In [22]:
## Fitting on the data
check_metrics(RandomForest_grid)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  9.8min finished


{'criterion': 'mae', 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
accuracy 0.8660760465434928
mae 17015.360273972605


**NOte:It shows that mean_absolute_error has improved from 17042 to 17015 thought the accuracy has decreased.**

## 2.GradientBoostingRegressor

In [24]:
## Making a gradientboosting hyperparameters grid 
np.random.seed(100)
gradientboost_grid={"criterion":["mae"],
           "n_estimators":[100,150,200],
           "min_samples_split":[2,4],
           "max_features":[None,"sqrt"]}
Gradient_grid=GridSearchCV(RandomForestRegressor(),
                               param_grid=gradientboost_grid,
                               cv=5,
                               verbose=True)

In [25]:
check_metrics(Gradient_grid)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  9.7min finished


{'criterion': 'mae', 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
accuracy 0.8660760465434928
mae 17015.360273972605


In [40]:
def check_metrics_new(model):
    model.fit(X_train,y_train)
    print("accuracy",model.score(X_valid,y_valid))
    print("mae",mean_absolute_error(y_valid,model.predict(X_valid)))

In [44]:
for i in range(590,650,10):
  print(f"{i} estimators")
  check_metrics_new(GradientBoostingRegressor(n_estimators=i))
  print("\n")

590 estimators
accuracy 0.8852450493791971
mae 15633.017280666898


600 estimators
accuracy 0.8827160527610369
mae 15800.072721166362


610 estimators
accuracy 0.8864951020326327
mae 15691.116060834514


620 estimators
accuracy 0.8882523848405276
mae 15621.247033954407


630 estimators
accuracy 0.8853231644387081
mae 15701.377759707291


640 estimators
accuracy 0.885152815048356
mae 15679.425202089096




**NOTE:For gradientboosting regressor ,the model with hypertuning n_estimators is better and 260 found to be the best**

## 3.DecisionTreeRegressor

In [26]:

np.random.seed(100)
decision_tree_grid={"criterion":['mae'],
                    "min_samples_split":[2,4,6,8,10],
                    "max_features":["auto","sqrt"],

                    }
Decision_grid=GridSearchCV(DecisionTreeRegressor(),
                                param_grid=decision_tree_grid,
                                cv=5,
                                verbose=5)

In [27]:
check_metrics(Decision_grid)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] criterion=mae, max_features=auto, min_samples_split=2 ...........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=mae, max_features=auto, min_samples_split=2, score=0.628, total=   0.2s
[CV] criterion=mae, max_features=auto, min_samples_split=2 ...........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  criterion=mae, max_features=auto, min_samples_split=2, score=0.693, total=   0.2s
[CV] criterion=mae, max_features=auto, min_samples_split=2 ...........


...

[CV]  criterion=mae, max_features=auto, min_samples_split=2, score=0.524, total=   0.2s
[CV] criterion=mae, max_features=auto, min_samples_split=2 ...........


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.9s remaining:    0.0s


[CV]  criterion=mae, max_features=auto, min_samples_split=2, score=0.777, total=   0.2s
[CV] criterion=mae, max_features=auto, min_samples_split=4 ...........
[CV]  criterion=mae, max_features=auto, min_samples_split=4, score=0.620, total=   0.2s
[CV] criterion=mae, max_features=auto, min_samples_split=4 ...........
[CV]  criterion=mae, max_features=auto, min_samples_split=4, score=0.707, total=   0.2s
[CV] criterion=mae, max_features=auto, min_samples_split=4 ...........
[CV]  criterion=mae, max_features=auto, min_samples_split=4, score=0.692, total=   0.2s
[CV] criterion=mae, max_features=auto, min_samples_split=4 ...........
[CV]  criterion=mae, max_features=auto, min_samples_split=4, score=0.530, total=   0.2s
[CV] criterion=mae, max_features=auto, min_samples_split=4 ...........
[CV]  criterion=mae, max_features=auto, min_samples_split=4, score=0.742, total=   0.2s
[CV] criterion=mae, max_features=auto, min_samples_split=6 ...........
[CV]  criterion=mae, max_features=auto, min_sa

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    6.0s finished


{'criterion': 'mae', 'max_features': 'auto', 'min_samples_split': 10}
accuracy 0.7392098140402124
mae 24701.532534246577


**NOTE:For decision tree regressor ,the model without hypertuning was better**

**FINAL NOTE:WITH ALL THOSE TUNING WE FOUND OUT THAT GradientBoostingRegressor IS THE BEST FOR THIS PROBLEM.**

# Train the model on full data 

In [46]:
##Splitting the data
X=train_preprocessed.drop("SalePrice",axis=1)
y=train_preprocessed["SalePrice"]

In [47]:
model=GradientBoostingRegressor(n_estimators=260).fit(X,y)

In [49]:
## Exporting the model
import pickle
pickle.dump(model,open("house_regression_model.pkl","wb"))

## Since while importing the model in pkl format in jupyter, it showed warning,we predicted on the test data here  by importing preprocessed test data**

In [50]:
## importing the test data
test=pd.read_csv("/content/drive/My Drive/test_preprocessed_house_regression.csv")
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,3,80.0,11622,2,0,4,4,1,5,1,13,2,3,1,3,5,6,1961,1961,2,1,11,13,3,0.0,4,5,2,4,4,4,5,468.0,4,144.0,270.0,882.0,1,5,2,4,896,0,0,896,0.0,0.0,1,0,2,1,4,5,7,0,0,2,1961.0,3,1.0,730.0,4,5,3,140,0,0,0,120,0,0,3,0,0,6,2010,9,5
1,1462,20,4,81.0,14267,2,0,1,4,1,1,1,13,3,3,1,3,6,6,1958,1958,4,1,12,14,2,108.0,4,5,2,4,4,4,1,923.0,6,0.0,406.0,1329.0,1,5,2,4,1329,0,0,1329,0.0,0.0,1,1,3,1,3,6,7,0,0,2,1958.0,3,1.0,312.0,4,5,3,393,36,0,0,0,0,0,0,1,12500,6,2010,9,5
2,1463,60,4,74.0,13830,2,0,1,4,1,5,1,9,3,3,1,5,5,5,1997,1998,2,1,11,13,3,0.0,4,5,3,3,4,4,3,791.0,6,0.0,137.0,928.0,1,3,2,4,928,701,0,1629,0.0,0.0,2,1,3,1,4,6,7,1,5,2,1997.0,1,2.0,482.0,4,5,3,212,34,0,0,0,0,0,3,0,0,3,2010,9,5
3,1464,60,4,78.0,9978,2,0,1,4,1,5,1,9,3,3,1,5,6,6,1998,1998,2,1,11,13,2,20.0,4,5,3,4,4,4,3,602.0,6,0.0,324.0,926.0,1,1,2,4,926,678,0,1604,0.0,0.0,2,1,3,1,3,7,7,1,3,2,1998.0,1,2.0,470.0,4,5,3,360,36,0,0,0,0,0,0,0,0,6,2010,9,5
4,1465,120,4,43.0,5005,2,0,1,2,1,5,1,23,3,3,5,3,8,5,1992,1992,2,1,7,7,3,0.0,3,5,3,3,4,4,1,263.0,6,0.0,1017.0,1280.0,1,1,2,4,1280,0,0,1280,0.0,0.0,2,0,2,1,3,5,7,0,0,2,1992.0,2,2.0,506.0,4,5,3,0,82,0,0,144,0,0,0,0,0,1,2010,9,5


##NOTE: TEST DATA PREPROCESSING IS DONE ON ANOTHER NOTEBOOK WHICH YOU CAN SEE IF YOU GO THROUGH THE REPOSITORY

## Predicting on the model

In [51]:
predictions=model.predict(test)
predictions

array([119158.68487262, 165505.87507234, 181130.61602463, ...,
       159005.88488545, 117619.24451931, 235075.3247618 ])

In [53]:
## Creating data frame 
submit_to_kaggle_colab=pd.DataFrame(columns=['Id','SalePrice'])
submit_to_kaggle_colab

,Id,SalePrice


In [54]:
submit_to_kaggle_colab["Id"],submit_to_kaggle_colab["SalePrice"]=test["Id"],predictions
submit_to_kaggle_colab

,Id,SalePrice
0,1461,119158.684873
1,1462,165505.875072
2,1463,181130.616025
3,1464,184503.432843
4,1465,183681.246909
...,...,...
1454,2915,72891.347321
1455,2916,75905.356421
1456,2917,159005.884885
1457,2918,117619.244519


In [55]:
## Exporting the data to be submitted in the kaggle
submit_to_kaggle_colab.to_csv("submit_to_kaggle_colab.csv",index=False)